In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import split
from pyspark.sql.functions import col, when 

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
df = spark.read.load("data/timesData.csv",
                     format="csv", sep=",", inferSchema="true", header="true")
df = df.select("world_rank", "university_name", "country", "total_score", "num_students", "international_students", "female_male_ratio", "year")
df = df.filter(df["total_score"] != "-")


In [34]:
split_col = split(df['female_male_ratio'], ':')
df = df.withColumn('female_ratio', split_col.getItem(0) / 100)
df = df.withColumn('male_ratio', split_col.getItem(1) / 100)
df = df.drop('female_male_ratio')

+----------+--------------------+--------------------+-----------+------------+----------------------+----+------------+----------+
|world_rank|     university_name|             country|total_score|num_students|international_students|year|female_ratio|male_ratio|
+----------+--------------------+--------------------+-----------+------------+----------------------+----+------------+----------+
|         1|  Harvard University|United States of ...|       96.1|      20,152|                   25%|2011|        null|      null|
|         2|California Instit...|United States of ...|       96.0|       2,243|                   27%|2011|        0.33|      0.67|
|         3|Massachusetts Ins...|United States of ...|       95.6|      11,074|                   33%|2011|        0.37|      0.63|
|         4| Stanford University|United States of ...|       94.3|      15,596|                   22%|2011|        0.42|      0.58|
|         5|Princeton University|United States of ...|       94.2|       7,9

In [39]:
filtro = df.groupBy(df["university_name"]).agg({"total_score":"avg"}).withColumnRenamed("avg(total_score)", "media")
filtro.show()

+----------+--------------------+--------------------+-----------+------------+----------------------+----+------------+----------+
|world_rank|     university_name|             country|total_score|num_students|international_students|year|female_ratio|male_ratio|
+----------+--------------------+--------------------+-----------+------------+----------------------+----+------------+----------+
|         1|  Harvard University|United States of ...|       96.1|      20,152|                   25%|2011|        null|      null|
|         2|California Instit...|United States of ...|       96.0|       2,243|                   27%|2011|        0.33|      0.67|
|         3|Massachusetts Ins...|United States of ...|       95.6|      11,074|                   33%|2011|        0.37|      0.63|
|         4| Stanford University|United States of ...|       94.3|      15,596|                   22%|2011|        0.42|      0.58|
|         5|Princeton University|United States of ...|       94.2|       7,9

In [42]:
filtro = filtro.withColumn("categoria", \
                                        when(col("media") >= 91, "A") \
                                       .when(col("media") >= 81, "B") \
                                       .when(col("media") >= 71, "C") \
                                       .when(col("media") >= 51, "D") \
                                       .otherwise("E"))

filtro.show()

+--------------------+------------------+---------+
|     university_name|             media|categoria|
+--------------------+------------------+---------+
| New York University| 70.03333333333333|        D|
|  Cornell University| 81.69999999999999|        B|
|University of St ...| 55.71666666666666|        D|
|University of Kon...| 46.86666666666667|        E|
|   Yonsei University| 46.73333333333333|        E|
|Pennsylvania Stat...| 62.68333333333333|        D|
|    Ghent University| 55.68333333333334|        D|
|Erasmus Universit...|56.666666666666664|        D|
|University of Col...|              52.3|        D|
|Scuola Superiore ...|              50.2|        E|
|University Colleg...|             49.72|        E|
|Newcastle University|              48.1|        E|
|Washington Univer...| 69.03333333333333|        D|
|Lancaster University|             52.25|        D|
|Stockholm University| 53.45000000000001|        D|
|  Indiana University|             50.92|        E|
|Queen’s Uni

In [ ]:
df = df.join(filtro, df.university_name == filtro.university_name, 'inner') \
.select(df.world_rank, df.university_name, df.country, df.total_score, df.num_students, df.international_students, df.female_ratio, df.male_ratio, df.year, filtro.media, filtro.categoria)

df.write.save("tabela.csv", format="csv")